In [2]:
from math314 import *

# 18 November 2019

In [16]:
df = pd.read_csv("https://raw.githubusercontent.com/roualdes/data/master/elmhurst.csv")
X = patsy.dmatrix("~ family_income", data=df)
yX = np.c_[df["gift_aid"], X]

In [28]:
betahat = optim(yX)
betas = pbootstrap(yX, 1001, optim, betahat)

NameError: name 'a' is not defined

In [ ]:
optim(yX)